KEYWORD EXTRACTION

In [66]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt') 

from rake_nltk import Rake
rake_nltk_var = Rake()
text = """frog and dog are playing in the forest"""
rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['playing', 'frog', 'forest', 'dog']


PARTS OF SPEECH TAGGING

In [67]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

tokenized = word_tokenize(text)

tagged = nltk.pos_tag(tokenized)
print(tagged)

[('frog', 'NN'), ('and', 'CC'), ('dog', 'NN'), ('are', 'VBP'), ('playing', 'VBG'), ('in', 'IN'), ('the', 'DT'), ('forest', 'NN')]


SENTIMENT ANALYSIS

In [68]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def retSetiment(text1):
    analyzer = SentimentIntensityAnalyzer()
    user_input = text1
    sentiment_scores = analyzer.polarity_scores(user_input)

    if sentiment_scores['compound'] >= 0.05:
        sentiment = "positive"
    elif sentiment_scores['compound'] <= -0.05:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    print(f"Sentiment: {sentiment}")
    return sentiment

ADDING SUBJECT DESCRIPTORS ( ENHANCERS )

In [69]:
import json
import re
from sentence_transformers import SentenceTransformer, util

def appendAdj(extNoun):
    extNoun=extNoun.lower()
    animals_adj = open("./data.json")
    ani_json = json.load(animals_adj)
    name_adj_dict = {}
    ani_json = ani_json['animals']
    sentSentiment = retSetiment(text)
    currAdj = []
    for i in ani_json:
        if(i['name'] == extNoun):
            for j in  range(len(i['adjectives'])):
                if(i['adjectives'][j]["type"] == sentSentiment):
                    currAdj.append(i['adjectives'][j]["value"])
    # print("current Adj : " , currAdj)
    name_adj_dict[extNoun] = currAdj
   
    return currAdj
    
    
    
allAdj = {}
allText = []

k=0

for i,j in (tagged):
    if(j=='NN'):
        allAdj[i] = appendAdj(i)
# for i in allAdj:
#     print(i," ",allAdj[i])
allText.append([])
allText[0]=[text]
length=1
for k in allAdj:
    allText.append([])
    if(allAdj[k] == []):
        continue
    for v1 in allAdj[k]:
        v1 += " " + k
        for m in allText[length-1]:
            # print(v1,end = "\n")
            allText[length].append(re.sub(k,v1,m))
            # print(allText[length])
    length+=1
for i in allText[length-1]:
    print(i+"\n")

Sentiment: positive
Sentiment: positive
Sentiment: positive
colorful frog and cute dog are playing in the forest

vibrant frog and cute dog are playing in the forest

striking frog and cute dog are playing in the forest

tropical frog and cute dog are playing in the forest

colorful frog and active dog are playing in the forest

vibrant frog and active dog are playing in the forest

striking frog and active dog are playing in the forest

tropical frog and active dog are playing in the forest

colorful frog and happy dog are playing in the forest

vibrant frog and happy dog are playing in the forest

striking frog and happy dog are playing in the forest

tropical frog and happy dog are playing in the forest

colorful frog and obedient dog are playing in the forest

vibrant frog and obedient dog are playing in the forest

striking frog and obedient dog are playing in the forest

tropical frog and obedient dog are playing in the forest



HABITAT MAPPING AND EMBEDDING

In [70]:
import json
import joblib

fdata = open('./data.json')
fdata = json.load(fdata)
fdata = fdata['animals']

def checkAndReturnHabitat(extNoun):
    for i in fdata:
        if(i['name'] == extNoun):
            # print(i['habitats'])
            return i['habitats']
    return []

def isHabitat(extNoun):
    model = joblib.load('habitatModel.pkl')
    vectorizer = joblib.load('vectorModel.pkl')

    user_input = extNoun

    user_input_preprocessed = user_input

    user_input_vec = vectorizer.transform([user_input_preprocessed])

    prediction = model.predict(user_input_vec)
    if prediction[0] == 1:
        return True
    return False


#habitat inclusion
currAllText = allText[length-1]
newAllText = []
setHabitats = []

flag = True
for i,j in (tagged):
    if (j == 'NN'):
        if(isHabitat(i) == False):
            flag = False
        else:
            flag = True
            break

#if there is no habitat found in the sentence
if flag == False:
    for i,j in (tagged):
        if ((j == 'NN') and (isHabitat(i) == False)): #dog
            setHabitats = checkAndReturnHabitat(i)
            if(setHabitats == []):
                continue
            for l in currAllText:
                for k in setHabitats:
                    temp = l
                    temp += ", in the " + k
                    newAllText.append(temp)

else:
    newAllText = currAllText
for i in newAllText:
    print(i,end = "\n")

colorful frog and cute dog are playing in the forest, in the rainforests
colorful frog and cute dog are playing in the forest, in the jungles
colorful frog and cute dog are playing in the forest, in the tropical forests
vibrant frog and cute dog are playing in the forest, in the rainforests
vibrant frog and cute dog are playing in the forest, in the jungles
vibrant frog and cute dog are playing in the forest, in the tropical forests
striking frog and cute dog are playing in the forest, in the rainforests
striking frog and cute dog are playing in the forest, in the jungles
striking frog and cute dog are playing in the forest, in the tropical forests
tropical frog and cute dog are playing in the forest, in the rainforests
tropical frog and cute dog are playing in the forest, in the jungles
tropical frog and cute dog are playing in the forest, in the tropical forests
colorful frog and active dog are playing in the forest, in the rainforests
colorful frog and active dog are playing in the 

FINDING COSINE SIMILARITY

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def cosineTop10(allText1):
    maxDict1 = {}
    top10Sentences = []
    for i in allText1:
        sentence1 = i
        sentence2 = text
        vectorizer = CountVectorizer().fit_transform([sentence1, sentence2])
        cosine_sim = cosine_similarity(vectorizer)
        # vectors1 = model.encode(col1, convert_to_tensor=True)
        # vectors2 = model.encode(col2, convert_to_tensor=True)
        # cosine_scores = util.cos_sim(vectors1, vectors2)
        if cosine_sim[0][1] >=0.5:
            maxDict1[i] = cosine_sim[0][1]
    l1 = 10
    ind1 = 0
    for k, v in sorted(maxDict1.items(), key=lambda item: item[1],reverse=True):
        if(ind1 >= l1 or ind1 >= len(maxDict1)):
            break
        ind1+=1
        top10Sentences.append(k)
    return top10Sentences 
topSentences = cosineTop10(newAllText)
for i in topSentences:
    print(i,end = "\n")
    

KNOWLEDGE GRAPH GENERATION - BASIC GoT

In [19]:
class Node:
    def __init__(self, name, node_type):
        self.name = name
        self.node_type = node_type
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def __str__(self) -> str:
        return f"{self.name} - ({self.node_type})"
   
class Graph:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, parent, child):
        parent.add_child(child)

    def get_children(self, node):
        return node.children
    def findNode(self,animalName):
        for i in self.nodes:
            if(i.node_type == 'animal' and i.name == animalName):
                temp =  self.get_children(i)
                return temp
        return []
# Create a graph
graph = Graph()
data = open('./data.json')

data_json = json.load(data)

data_json = data_json['animals']

root = Node("animal", "root")
graph.add_node(root)
for i in range(len(data_json)):
    n1 = Node(data_json[i]['name'],"animal")
    graph.add_node(n1)
    for adj in data_json[i]["adjectives"]:
        adjective_node = Node(adj, "adjective")
        graph.add_node(adjective_node)
        graph.add_edge(n1, adjective_node)

    for habitat in data_json[i]["habitats"]:
        habitat_node = Node(habitat, "habitat")
        graph.add_node(habitat_node)
        graph.add_edge(n1, habitat_node)

testAnimal = "dog"
cdren = graph.findNode(testAnimal)
if(cdren != []):
    for i in cdren:
        print(i)
else:
    print("Not Found")

{'value': 'cute', 'type': 'positive'} - (adjective)
{'value': 'angry', 'type': 'negative'} - (adjective)
{'value': 'ferocious', 'type': 'negative'} - (adjective)
{'value': 'active', 'type': 'positive'} - (adjective)
{'value': 'happy', 'type': 'positive'} - (adjective)
{'value': 'obedient', 'type': 'positive'} - (adjective)
rainforests - (habitat)
jungles - (habitat)
tropical forests - (habitat)


Follow Up:
    - image type - photo, sketch, cartoon...
    - camera features - focus, lens...
    - time of the scenario - morning, afternoon, night, dinner...
    - adding +,- weights

In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load your dataset
data = pd.read_csv('habitat.csv')
# print(data)
# Split data into features (X) and labels (y)
X = data['text']
y = data['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform the text data into numerical features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Choose a classification model (e.g., Logistic Regression)
classifier = LogisticRegression()

# Train the model
classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

# Use the trained model to predict if a new word is a habitat or not
new_word = "Forest"
new_word_tfidf = tfidf_vectorizer.transform([new_word])
prediction = classifier.predict(new_word_tfidf)

if prediction[0] == 1:
    print(f"{new_word} is a habitat.")
else:
    print(f"{new_word} is not a habitat.")


Accuracy: 0.8787878787878788
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.75      0.84        28
           1       0.84      0.97      0.90        38

    accuracy                           0.88        66
   macro avg       0.90      0.86      0.87        66
weighted avg       0.89      0.88      0.88        66

Forest is not a habitat.
